In [1]:
# Loading the required packages
import csv
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from lxml import html
try:
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen
import dateutil
from dateutil.parser import *
from datetime import datetime
import pathlib
from htmldate import find_date
import justext
import requests
from newspaper import fulltext
import codecs
import html2text
import trafilatura
import spacy
import dframcy
from dframcy import DframCy
import pickle
import chardet
import urllib

In [66]:
###### ENGLAND ######

In [5]:
# Defining the function to read in all the html files, clean the text, get further publishing dates and select the earliest one, and store everything into csv
# process statewise UK VERSION --> type "General"

def process_state_UK(state):

    # compiling the regex search terms
    agile_regex = re.compile(r'\bagility\b|\bagile\b', re.IGNORECASE | re.UNICODE) # for agile
    agile_context_regex = re.compile(r'\s*([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+agil.*?\s+([^\s]+)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+', re.IGNORECASE | re.UNICODE) # for context of the first mention
    digital_regex = re.compile(r'\bdigital\b|\bditialization\b|\bdigitalisation\b\bdigital transformation\b', re.IGNORECASE | re.UNICODE) # for anything related to digital

    # getting the csv which was produced by scrapy as input file
    csv_main = csv.DictReader(open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/CSVs/UK/agile_sites_output_UK_{}.csv".format(state)), fieldnames=["id", 'url', 'domain', 'date1', 'date2', 'date3', 'heading']) 

    # creating an empty array to write into in the processing loop
    array = {}

    # getting every line of the input csv into the array
    for line in csv_main: 
        array[line["id"]] = line
    
    # specfifying the the directory where the text files should be stored
    text_dir = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/TextFiles/UK/{}".format(state)
    
    # making the directory if it doesn't exist
    pathlib.Path(text_dir).mkdir(parents=True, exist_ok=True)
    
    # specifying variables to keep track of the progress while processing
    total_count = len(array)
    processed = 0

    deleted_cases = []

    # this is the main working loop
    for id, line in array.items():   
        # print function to keep track of which files are being processed     
        print("processing {}".format(id))
        # getting the id from the array
        id = line["id"]
        # getting the domain from the array
        domain = line["domain"]
        # getting the url from the array
        url = line["url"]
        # specifying the path where the html to process is located
        path = '/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/HTMLs/UK/{}/{}/{}.html'.format(state, domain, id)
        # opening the html file while catching an encoding error     
        try:  
            soup = BeautifulSoup(open(path), "html.parser")
        except UnicodeDecodeError:
            try:
                rawdata = open(file=path, mode="rb").read()
                encoding = chardet.detect(rawdata)['encoding']
                print('Catching encoding error: ', chardet.detect(rawdata))
                soup = BeautifulSoup(open(path, encoding=encoding), "html.parser")    
            except UnicodeDecodeError:
                deleted_cases.append({'id' : id, 'domain':domain, 'url':url, 'path':path})
                print('deleting: ', id)
                continue

    

        ### In the following further potential publication dates are parsed and the oldest one is select as "final date" ###

        # Getting date4
        date4_element = soup.select_one("span.date")
        date4 = ""
        if date4_element is not None:
            date4 = date4_element.get_text()
        
        # Getting date6
        date6_element = soup.select_one("h1#page-title + p")
        date6 = ""
        if date6_element is not None:
            date6 = date6_element.get_text()

       # Getting date5
        htmlparser = etree.HTMLParser()    
        try:
            tree = etree.parse(open(path), htmlparser)
        except UnicodeDecodeError:
            tree = etree.parse(open(path, encoding=encoding), htmlparser)


        date5 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 23)")

        # a special cases for date 5
        date5_1 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 25)")


        # Getting date7 from National Archives --> the date the site was archived on --> the date the site was released on would have been even earlier
        date7 = tree.xpath("substring(substring-after(/html/head/script/text(), 'timestamp'),9,14)")

  
        # different parsing for for htmldate
        try:
            html_tree = html.parse(open(path))
        except UnicodeDecodeError:
            html_tree = html.parse(open(path, encoding=encoding))
           


        # Getting date with the htmldate package
        htmldate = find_date(html_tree)


        # since html date gives out nonetype when it doesn't find anything, it has to be respecified to empty string so that it works with dateparser later on
        if htmldate is None:
            htmldate = ''

        #htmldate = htmldate.astype('str')



        # assinging the already succesfully parsed dates during crawling stage
        date1 = line["date1"]
        date2 = line["date2"]


        # Storing the oldest date as final date variable in python date format
        date_vars = [date1, date2, date4, date5, date5_1, date6, date7, htmldate]
        
        final_date = None 

        for date_var in date_vars:
            try:
                if final_date is None:
                    final_date = parse(date_var, ignoretz = True)
                elif parse(date_var, ignoretz=True) < final_date:
                    final_date = parse(date_var, ignoretz=True)
            except dateutil.parser._parser.ParserError:
                pass 



        #deleting all non-body text content from the html
        
        # defining the html elements to delete
        for script in soup(['script', 'style', 'meta', 'a', 'head', 'footer', 'navbar', 'header', 'search-box', 'global-cookie-message', 'id="global-cookie-message"', 'global-bar', 'menu', 'noscript', 'global-cookie-message', 'search']):
            script.decompose()    # deleting those elements
            
        # geting text
        doctext = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in doctext.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        doctext = '\n'.join(chunk for chunk in chunks if chunk)
        
  

        # saving the cleaned text into the directory
        with open("{}/{}.txt".format(text_dir, id), "w") as textfile:
            textfile.write(doctext)
        

 
        #### In the following the matches for the search terms (agile, digital) are gathered ####
        
        # finding the matches for agile and agility
        agile_term = []
        agile_term = agile_regex.findall(doctext.lower())
    

        # finding the matches for digital
        digital_term = []
        digital_term = digital_regex.findall(doctext.lower())


        # finding the context for agile
        agile_context = []
        agile_context = agile_context_regex.search(doctext)

        agile_context_pre = ""
        agile_context_post = ""
        
        if agile_context is not None:
            agile_context_pre = " ".join(agile_context.group(1,2,3,4))
            agile_context_post = " ".join(agile_context.group(5,6,7,8))
        

        # assigning all generated variables as line items to the array

        if len(agile_term) == 0:
            line["agile_term"] = "" 
        else:
            line["agile_term"] = ",".join(agile_term)

        if len(digital_term) == 0:
            line["digital_term"] = "" 
        else:
            line["digital_term"] = ",".join(digital_term)


        line["agile_context_pre"] = agile_context_pre
        line["agile_context_post"] = agile_context_post 
        line["date4"] = date4
        line["date5"] = date5
        line["date5_1"] = date5_1
        line["date6"] = date6
        line["date7"] = date7
        line["final_date"] = final_date
        line["htmldate"] = htmldate
        line["country"] = "UK"
        line["level"] = state
        line["text_file_loc"] = "{}/{}.txt".format(text_dir, id)
        line['doctext'] = doctext

        
        # Printing processing status
        processed += 1
        print("processed: {}%".format((processed/total_count)*100))
        print("Current Time: ", datetime.now())

    # opening the output csv
    outputfile = open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/{}.csv".format(state), "w")
    
    # writing all the array lines into the csv file
    writer = csv.DictWriter(outputfile, fieldnames=["id", "country", "level", 'url', 'domain', 'date1', 'date2', 'date3', 'date4', 'date5', 'date5_1', 'date6', 'date7', 'htmldate', 'final_date', 'heading', 'agile_term', 'agile_context_pre', 'agile_context_post', 'digital_term', 'text_file_loc', 'doctext',])
    writer.writeheader()
    for id, line in array.items():
        writer.writerow(line)
    outputfile.close()  

    # saving the ones that have been deleted
    df_deleted_cases = pd.DataFrame(deleted_cases)

    df_deleted_cases.to_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/{}_deleted_cases.csv".format(state), header=True)

In [6]:
#Running processing for UK
"""
process_state_UK("General")


6732-eab9-3a73-9e33-7c6604384c49
processed: 97.15211459754434%
Current Time:  2020-04-16 04:24:12.192759
processing 6cf0635d-a05c-3ad0-8284-8262dd3f0072
processed: 97.16916780354707%
Current Time:  2020-04-16 04:24:12.356506
processing 2af7cf6d-3153-3485-ab15-e45bb8c165cb
processed: 97.1862210095498%
Current Time:  2020-04-16 04:24:12.522390
processing 1c1d8752-011c-3081-94ae-58479b0c0ce2
processed: 97.20327421555253%
Current Time:  2020-04-16 04:24:12.806708
processing 0d99d625-2fab-3e51-8f3a-2c3e4d77d2bc
processed: 97.22032742155525%
Current Time:  2020-04-16 04:24:12.976138
processing c5a403ff-124a-3d4d-808b-6fc744c01627
processed: 97.23738062755798%
Current Time:  2020-04-16 04:24:13.160625
processing 50304460-eb7c-36f5-ab39-cda6b6b812b9
processed: 97.25443383356071%
Current Time:  2020-04-16 04:24:13.333621
processing 553c660d-7888-3998-b2a4-8f7377aacfea
processed: 97.27148703956344%
Current Time:  2020-04-16 04:24:13.511875
processing a482dead-6c88-3580-9906-2cd0eb5e2623
processe

In [2]:
# Praparing the cleaning algorithm

# Writing Spacy Lemmatizer for lists function
def spacy_list_lemmatizer(text, nlp):
    nlp.disable_pipes('tagger', 'ner')
    doclist = list(nlp.pipe(text, n_threads=3,  batch_size=50))
    docs=[]
    for i, doc in enumerate(doclist):
        docs.append(' '.join([listitem.lemma_ for listitem in doc]))
    return docs


# creating the lemmas of all english co-searchterms
searchterms_eng = ["'agile' capability", "'agile' ceremony", "'agile' certificate", "'agile' certification", "'agile' coach", "'agile' coaching", "'agile' delivery", "'agile' development", "'agile' digital capability ", "'agile' environment", "'agile' innovation", "'agile' management", "'agile' manifesto", "'agile' method", "'agile' methodology", "'agile' mindset", "'agile' operating model", "'agile' organisation", "'agile' organization unit", "'agile' principles", "'agile' processes", "'agile' programming", "'agile' project", "'agile' project management", "'agile' software development", "'agile' stages", "'agile' structures", "'agile' team", "'agile' techniques", "'agile' transformation", "'agile' transition", "'agile' Value", "'agile' walls", "'agile' way", "'agile' way of working", "'agile' work", "'agile' work practices", "'agile' working and management methods", "'agile' working culture", "'agile' working methods", "agile capability", "agile ceremony", "agile certificate", "agile certification", "agile coach", "agile coaching", "agile delivery", "agile development", "agile digital capability ", "agile environment", "agile innovation", "agile management", "agile manifesto", "agile method", "agile methodology", "agile mindset", "agile operating model", "agile organisation", "agile organization unit", "agile principles", "agile programming", "agile project", "agile project management", "agile software development", "agile stages", "agile structures", "agile team", "agile techniques", "agile transformation", "agile transition", "agile value", "agile walls", "agile way", "agile way of working", "agile work", "agile work practices", "agile working and management methods", "agile working culture", "agile working methods", "'agile' capability", "'agile' ceremony", "'agile' certificate", "'agile' certification", "'agile' coach", "'agile' coaching", "'agile' delivery", "'agile' development", "'agile' digital capability ", "'agile' environment", "'agile' innovation", "'agile' management", "'agile' manifesto", "'agile' method", "'agile' methodology", "'agile' mindset", "'agile' operating model", "'agile' organisation", "'agile' organization unit", "'agile' principles", "'agile' processes", "'agile' programming", "'agile' project", "'agile' project management", "'agile' software development", "'agile' stages", "'agile' structures", "'agile' team", "'agile' techniques", "'agile' transformation", "'agile' transition", "'agile' Value", "'agile' walls", "'agile' way", "'agile' way of working", "'agile' work", "'agile' work practices", "'agile' working and management methods", "'agile' working culture", "'agile' working methods", "backlog", "co-design", "collaboration spaces", "collaborative design", "cross-departmental approach", "daily scrum", "definition of done", "definition of ready", "design thinking", "development team", "empathy map", "extreme user", "fail fast", "fail forward", "free of hierarchy", "hierarchy free space", "innovation culture", "innovation management", "iteration", "iterative", "Kanban", "kanban board", "lean software development", "lean start up", "lean start-up", "lean startup", "minimum viable product", "MVP", "new team and management structures", "new work", "new working methods", "new working models", "planning poker", "product backlog", "product owner", "project canvas", "project work", "scrum", "scrum coach", "scrum master", "scrum team", "service design", "short-cycle results", "software development", "sprint", "sprint backlog", "sprint planning", "sprint retrospective", "sprint review", "time box", "timeboxing", "user needs", "user requirements", "user research", "user story", "user testing", "user-centred", "user-centred design", "user-friendly solutions", "user-oriented", "visual thinking", "whiteboard", "wicked problem", "wireframe"]

nlp_eng = spacy.load("en_core_web_sm")

searchterms_eng_lemma = spacy_list_lemmatizer(searchterms_eng, nlp_eng)

searchterms_eng_lemma = [each_string.lower() for each_string in searchterms_eng_lemma]

# creating the lemmas of all German and English co-searchterms for German sites, since English terms might also appear on German sites
searchterms_de_eng = ["abteilungsübergreifender Ansatz", "agil arbeitende Organisationseinheit", "agil arbeitende Teams", "agile Arbeitsformen", "agile Arbeitskultur", "agile Arbeitsmethoden", "agile Arbeitspraktiken", "agile Arbeitsumgebung", "agile Arbeitsweise", "agile Bereitstellung", "agile digitale Fähigkeit ", "agile Entwicklung", "agile Fähigkeiten ", "agile Innovation", "agile Methode", "agile Methodik", "agile Organisation", "agile Phasen", "agile Prinzipien", "agile Prozesse", "agile Softwareentwicklung", "agile Softwareweiterentwicklung", "agile Strukturen", "agile Techniken", "agile Transformation", "Agile Transition", "agile Wände", "agile Weise", "agile Zeremonie", "agile Zertifizierung", "agilen Arbeits- und Managementmethoden", "agiler Ansatz", "agiler Coach", "agiler Wert", "agiles Arbeiten", "agiles Betriebsmodell", "agiles Coaching", "agiles Management", "Agiles Manifest", "agiles Mindset", "agiles Programmieren", "agiles Projekt", "Agiles Projektmanagement", "agiles Team", "agiles Training ", "agiles Zertifikat", '"agil" arbeitende Organisationseinheit', '"agil" arbeitende Teams', '"agile" Arbeitsformen', '"agile" Arbeitskultur', '"agile" Arbeitsmethoden', '"agile" Arbeitspraktiken', '"agile" Arbeitsumgebung', '"agile" Arbeitsweise', '"agile" Bereitstellung', '"agile" digitale Fähigkeit ', '"agile" Entwicklung', '"agile" Fähigkeiten ', '"agile" Innovation', '"agile" Methode', '"agile" Methodik', '"agile" Organisation', '"agile" Phasen', '"agile" Prinzipien', '"agile" Prozesse', '"agile" Softwareentwicklung', '"agile" Softwareweiterentwicklung', '"agile" Strukturen', '"agile" Techniken', '"agile" Transformation', '"agile" Transition', '"agile" Wände', '"agile" Weise', '"agile" Zeremonie', '"agile" Zertifizierung', '"agilen" Arbeits- und Managementmethoden', '"agiler" Ansatz', '"agiler" Coach', '"agiler" Wert', '"agiles" Arbeiten', '"agiles" Betriebsmodell', '"agiles" Coaching', '"agiles" Management', '"agiles" Manifest', '"agiles" Mindset', '"agiles" Programmieren', '"agiles" Projekt', '"agiles" Projektmanagement', '"agiles" Team', '"agiles" Training ', '"agiles" Zertifikat', "'agil' arbeitende Organisationseinheit", "'agil' arbeitende Teams", "'agile' Arbeitsformen", "'agile' Arbeitskultur", "'agile' Arbeitsmethoden", "'agile' Arbeitspraktiken", "'agile' Arbeitsumgebung", "'agile' Arbeitsweise", "'agile' Bereitstellung", "'agile' digitale Fähigkeit ", "'agile' Entwicklung", "'agile' Fähigkeiten ", "'agile' Innovation", "'agile' Methode", "'agile' Methodik", "'agile' Organisation", "'agile' Phasen", "'agile' Prinzipien", "'agile' Prozesse", "'agile' Softwareentwicklung", "'agile' Softwareweiterentwicklung", "'agile' Strukturen", "'agile' Techniken", "'agile' Transformation", "'agile' Transition", "'agile' Wände", "'agile' Weise", "'agile' Zeremonie", "'agile' Zertifizierung", "'agilen' Arbeits- und Managementmethoden", "'agiler' Ansatz", "'agiler' Coach", "'agiler' Wert", "'agiles' Arbeiten", "'agiles' Betriebsmodell", "'agiles' Coaching", "'agiles' Management", "'agiles' Manifest", "'agiles' Mindset", "'agiles' Programmieren", "'agiles' Projekt", "'agiles' Projektmanagement", "'agiles' Team", "'agiles' Training ", "'agiles' Zertifikat", "Backlog", "Co-Design", "Daily Scrum", "Definition of Done", "Definition of Ready", "Design Thinking", "Empathy Map", "Entwicklungsteam", "Extremnutzer Nutzer", "fail fast", "fail forward", "hierarchiefrei", "hierarchiefreien Raum", "Innovationskultur", "Innovationsmanagement", "Iteration", "iterativ ", "Kanban", "Kanbanboard", "Kollaborationsräume", "kollaboratives Design", "kurzzyklische Ergebnisse", "Lean Start up", "Lean Start-up", "Lean Startup",  "Minimum Viable Product", "MVP", "neue Arbeitsmodelle", "neue Arbeitsweisen", "neue Team- und Führungsstrukturen", "New Work", "Nutzeranforderungen", "Nutzerbedürfnisse", "Nutzerforschung", "nutzerorientiert", "nutzerorientierte Lösungen", "Nutzerstory", "Nutzertest", "nutzerzentriert", "nutzerzentriertes Design", "Planning Poker", "Product Backlog", "Product Owner", "Project Canvas", "Projektarbeiten", "schlanke Softwareentwicklung", "Scrum", "Scrum Coach", "Scrum Master", "Scrum Team", "Service Design", "Softwareentwicklung", "Sprint", "Sprint Backlog", "Sprint Planning", "Sprint Retrospective", "Sprint Review", "Timebox", "Timeboxing", "visuelles Denken", "Whiteboard", "Wicked Problem", "Wireframe", "'agile' capability", "'agile' ceremony", "'agile' certificate", "'agile' certification", "'agile' coach", "'agile' coaching", "'agile' delivery", "'agile' development", "'agile' digital capability ", "'agile' environment", "'agile' innovation", "'agile' management", "'agile' manifesto", "'agile' method", "'agile' methodology", "'agile' mindset", "'agile' operating model", "'agile' organisation", "'agile' organization unit", "'agile' principles", "'agile' processes", "'agile' programming", "'agile' project", "'agile' project management", "'agile' software development", "'agile' stages", "'agile' structures", "'agile' team", "'agile' techniques", "'agile' transformation", "'agile' transition", "'agile' Value", "'agile' walls", "'agile' way", "'agile' way of working", "'agile' work", "'agile' work practices", "'agile' working and management methods", "'agile' working culture", "'agile' working methods", "agile capability", "agile ceremony", "agile certificate", "agile certification", "agile coach", "agile coaching", "agile delivery", "agile development", "agile digital capability ", "agile environment", "agile innovation", "agile management", "agile manifesto", "agile method", "agile methodology", "agile mindset", "agile operating model", "agile organisation", "agile organization unit", "agile principles", "agile processes", "agile programming", "agile project", "agile project management", "agile software development", "agile stages", "agile structures", "agile team", "agile techniques", "agile transformation", "agile transition", "agile Value", "agile walls", "agile way", "agile way of working", "agile work", "agile work practices", "agile working and management methods", "agile working culture", "agile working methods", "'agile' capability", "'agile' ceremony", "'agile' certificate", "'agile' certification", "'agile' coach", "'agile' coaching", "'agile' delivery", "'agile' development", "'agile' digital capability ", "'agile' environment", "'agile' innovation", "'agile' management", "'agile' manifesto", "'agile' method", "'agile' methodology", "'agile' mindset", "'agile' operating model", "'agile' organisation", "'agile' organization unit", "'agile' principles", "'agile' processes", "'agile' programming", "'agile' project", "'agile' project management", "'agile' software development", "'agile' stages", "'agile' structures", "'agile' team", "'agile' techniques", "'agile' transformation", "'agile' transition", "'agile' Value", "'agile' walls", "'agile' way", "'agile' way of working", "'agile' work", "'agile' work practices", "'agile' working and management methods", "'agile' working culture", "'agile' working methods", "backlog", "co-design", "collaboration spaces", "collaborative design", "cross-departmental approach", "daily scrum", "definition of done", "definition of ready", "design thinking", "development team", "empathy map", "extreme user", "fail fast", "fail forward", "free of hierarchy", "hierarchy free space", "innovation culture", "innovation management", "iteration", "iterative", "Kanban", "kanban board", "lean software development", "lean start up", "lean start-up", "lean startup", "minimum viable product", "MVP", "new team and management structures", "new work", "new working methods", "new working models", "planning poker", "product backlog", "product owner", "project canvas", "project work",  "scrum", "scrum coach", "scrum master", "scrum team", "service design", "short-cycle results", "software development", "sprint", "sprint backlog", "sprint planning", "sprint retrospective", "sprint review", "time box", "timeboxing", "user needs", "user requirements", "user research", "user story", "user testing", "user-centred", "user-centred design", "user-friendly solutions", "user-oriented", "visual thinking", "whiteboard", "wicked problem", "wireframe"]

nlp_de = spacy.load("de_core_news_sm")

# lemmatizing
searchterms_de_eng_lemma = spacy_list_lemmatizer(searchterms_de_eng, nlp_de)

# making lowercase
searchterms_de_eng_lemma = [each_string.lower() for each_string in searchterms_de_eng_lemma]

# deleting duplicates
searchterms_de_eng_lemma = list(set(searchterms_de_eng_lemma)) 

with open("searchterms_de_eng_lemma.pickle", "wb") as file:
    pickle.dump(searchterms_de_eng_lemma, file)

with open("searchterms_eng_lemma.pickle", "wb") as file:
    pickle.dump(searchterms_eng_lemma, file)

In [4]:
# Cleaning the data and selecting only relevant pages based on the co-searchterms dictionary

inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/General.csv"
outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/General_cleaned.csv"

# loading the spacy model
nlp = spacy.load("en_core_web_sm")
# switching off irrelevant spacy functions
nlp.disable_pipes('tagger', 'ner')

# defining the keywords search function
def searchfunction(row):
    matchstring = ""
    count = 0
    for term in searchterms_eng_lemma:
        if term.lower() in row['doctext_lemma']:
            count += 1
            matchstring = matchstring + term.lower() + ", "
    row['co_searchterms_nr'] = count
    row['co_searchterms_matches'] = matchstring
    return row


# opening and cleaning / preprocessing dataframe
with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
    # reading the csv
    df = pd.read_csv(inFile)
    # sorting the values by final date
    df.sort_values(by=['final_date'])
    # dropping all sites which didn't yield any match for the agile term
    df.dropna(axis=0, how='any', thresh=None, subset=(['agile_term']), inplace=True)
    # dropping potential duplicates based on the heading and the context of the first agile term mention
    df.drop_duplicates(subset=(['heading', 'agile_context_pre', 'agile_context_post']), inplace=True)
    # deleting sites with csv previews
    df = df[~df.url.str.endswith("csv/preview")]
    # deleting search sites
    df = df[~df.url.str.contains('/search/')]
    # deleting supplier sites
    df = df[~df.url.str.contains('supplier')]
    # deleting agile metioned in a tweet that occurs on multiple sites 
    df['doctext'] = df['doctext'].map(lambda x: x.strip('#Agile Development lessons learned from Gov.uk'))
    #deleting cases where no date has been found (htmldate attributes 2020-01-01 for no dates)
    df = df[df['final_date'] != '2020-01-01 00:00:00']
    #deleting all cases from 2020 because the analysis focuses on everything up until 2019
    df = df[df['final_date'] < '2020-01-01 00:00:00']
    # changing the final_date column to a pandas datetime object
    df['final_date'] = pd.to_datetime(df['final_date'])

    # lemmatizing the text
    lemma = []
    for doc in nlp.pipe(df['doctext'].astype('unicode').values, batch_size=50, n_threads=3):
        if doc.is_parsed:
            lemma.append([n.lemma_ for n in doc])
        else:
            lemma.append(None)

    # storing the lemmas in a new column 
    df['doctext_lemma'] = lemma
    
    #making lemmas lowercase
    df['doctext_lemma'] = df['doctext_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
    #joining list of lemmas into one string agein, so it becomes matchable by the searchfunction
    df['doctext_lemma'] = df['doctext_lemma'].map(lambda lemmas: " ".join(lemmas))
 
    #applying the keyword search function 
    df = df.apply(searchfunction, axis = 1)

    # delete everything that didn't yield a co-searchterm match
    df = df[df['co_searchterms_nr'] != 0 ]
    df.to_csv(outFile)

In [10]:
# Adding Organisation Names based on domain

# specifying input and output file and paths
inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/General_cleaned.csv"
outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/General_cleaned_org.csv"

# file with all the uk subdomains (manual eddited to include subsubdomains)
domains_dict = pd.read_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/List_of_gov.uk_domain_names_as_at_28_Oct_2019_manualEdit.csv")

# compiling regex to extract domains from url (also in the case of a site on national archives)
domain_regex = re.compile(r'http(s?)://(?!.*http)(www\.)?([a-zA-Z0-9\-.]+)', re.IGNORECASE | re.UNICODE)

# defining the url extraction function
def extract(url):
    match = domain_regex.search(url)
    if match:
        match_domain = match.group(3)
        return '.'.join(match_domain.split('.')[-4:])
    else:
        print(url)
        return ''

# adding the domain / organisation names to the file
with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
    df = pd.read_csv(inFile)
    # getting the subdomain from 
    df['sub_domain'] = df.apply(lambda row: extract(row['url']), axis=1)
    # merging the file with the dataframe that has all the subdomains and organisation names
    df2 = pd.merge(df, domains_dict, left_on = 'sub_domain', right_on='Domain: Domain Name', how='left')
    # saving the file
    df2.to_csv(outFile)

In [72]:
######### GERMANY ############

In [3]:
# Defining the function to read in all the html files, clean the text, get further publishing dates and select the earliest one, and store everything into csv
def process_state_DE(state):

    # compiling the regex search terms
    agile_regex = re.compile(r'\bagility\b|\bagil\w{0,2}\b|\bagilität\b', re.IGNORECASE | re.UNICODE) # for agil keyword
    agile_context_regex = re.compile(r'\s*([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+agil.*?\s+([^\s]+)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+', re.IGNORECASE | re.UNICODE) # to get the context
    digital_regex = re.compile(r'\bdigital\w{0,2}\b|\bdigitalisierung\b|\bdigitale transformation\b', re.IGNORECASE | re.UNICODE) # to get everything related digitalisation

    # getting the csv which was produced by scrapy as input file
    csv_main = csv.DictReader(open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/CSVs/Germany/agile_sites_output_Germany_{}.csv".format(state)), fieldnames=["id", 'url', 'domain', 'date1', 'date2', 'date3', 'heading'])

    # creating an empty array to write into in the processing loop
    array = {}

    # getting every line of the input csv into the array
    for line in csv_main: 
        array[line["id"]] = line
    
    # specfifying the the directory where the text files should be stored
    text_dir = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/TextFiles/Germany/{}".format(state)
    
    # making the directory if it doesn't exist
    pathlib.Path(text_dir).mkdir(parents=True, exist_ok=True)
    
    # specifying variables to keep track of the progress while processing
    total_count = len(array)
    processed = 0

    # this is the main working loop
    for id, line in array.items():   
        # print function to keep track of which files are being processed     
        print("processing {}".format(id))
        # getting the id from the array
        id = line["id"]
        # getting the domain from the array
        domain = line["domain"]
        # getting the url from the array
        url = line["url"]

        #excluding hits from berlin jobs which are a lot and are de facto a germany wide job portal, which is of no relevance for this research 
        if domain == 'jobs.berlin.de':
            continue
            
        # specifying the path where the html to process is located
        path = '/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/1_Data_Collection/DATA/HTMLs/Germany/{}/{}/{}.html'.format(state, domain, id)
        # opening the html file while catching an encoding error     
        try:  
            soup = BeautifulSoup(open(path), "html.parser")
        except UnicodeDecodeError:
            rawdata = open(file=path, mode="rb").read()
            encoding = chardet.detect(rawdata)['encoding']
            print('Catching encoding error: ', chardet.detect(rawdata))
            soup = BeautifulSoup(open(path, encoding=encoding), "html.parser")    
        
        ### In the following further potential publication dates are parsed and the oldest one is select as "final date" ###

        # Getting date4
        date4_element = soup.select_one("span.date")
        date4 = ""
        if date4_element is not None:
            date4 = date4_element.get_text()
        
        # Getting date6
        date6_element = soup.select_one("h1#page-title + p")
        date6 = ""
        if date6_element is not None:
            date6 = date6_element.get_text()

       # Getting date5
        htmlparser = etree.HTMLParser()    
        try:
            tree = etree.parse(open(path), htmlparser)
        except UnicodeDecodeError:
            tree = etree.parse(open(path, encoding=encoding), htmlparser)
        
        date5 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 23)")

        # a special cases for date 5
        date5_1 = tree.xpath("substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 25)")


        # Getting date7 from National Archives --> the date the site was archived on --> the date the site was released on would have been even earlier
        date7 = tree.xpath("substring(substring-after(/html/head/script/text(), 'timestamp'),9,14)")

  
        # different parsing for for htmldate
        try:
            html_tree = html.parse(open(path))
        except UnicodeDecodeError:
            html_tree = html.parse(open(path, encoding=encoding))
            

        # Getting date with the htmldate package
        htmldate = find_date(html_tree)


        # since html date gives out nonetype when it doesn't find anything, it has to be respecified to empty string so that it works with dateparser later on
        if htmldate is None:
            htmldate = ''

        #htmldate = htmldate.astype('str')



        # assinging the already succesfully parsed dates during crawling stage
        date1 = line["date1"]
        date2 = line["date2"]


        # Storing the oldest date as final date variable in python date format
        date_vars = [date1, date2, date4, date5, date5_1, date6, date7, htmldate]
        
        final_date = None 

        for date_var in date_vars:
            try:
                if final_date is None:
                    final_date = parse(date_var, ignoretz = True)
                elif parse(date_var, ignoretz=True) < final_date:
                    final_date = parse(date_var, ignoretz=True)
            except dateutil.parser._parser.ParserError:
                pass 



        #deleting all non-body text content from the html
        
        # defining the html elements to delete
        for script in soup(['script', 'style', 'meta', 'a', 'head', 'footer', 'navbar', 'header', 'search-box', 'global-cookie-message', 'id="global-cookie-message"', 'global-bar', 'menu', 'noscript', 'global-cookie-message', 'search']):
            script.decompose()    # deleting those elements
            
        # geting text
        doctext = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in doctext.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        doctext = '\n'.join(chunk for chunk in chunks if chunk)
        
  

        # saving the cleaned text into the directory
        with open("{}/{}.txt".format(text_dir, id), "w") as textfile:
            textfile.write(doctext)
        

 
        #### In the following the matches for the search terms (agile, digital) are gathered ####
        
        # finding the matches for agile and agility
        agile_term = []
        agile_term = agile_regex.findall(doctext.lower())
    

        # finding the matches for digital
        digital_term = []
        digital_term = digital_regex.findall(doctext.lower())


        # finding the context for agile
        agile_context = []
        agile_context = agile_context_regex.search(doctext)

        agile_context_pre = ""
        agile_context_post = ""
        
        if agile_context is not None:
            agile_context_pre = " ".join(agile_context.group(1,2,3,4))
            agile_context_post = " ".join(agile_context.group(5,6,7,8))
        

        # assigning all generated variables as line items to the array

        if len(agile_term) == 0:
            line["agile_term"] = "" 
        else:
            line["agile_term"] = ",".join(agile_term)

        if len(digital_term) == 0:
            line["digital_term"] = "" 
        else:
            line["digital_term"] = ",".join(digital_term)


        line["agile_context_pre"] = agile_context_pre
        line["agile_context_post"] = agile_context_post 
        line["date4"] = date4
        line["date5"] = date5
        line["date5_1"] = date5_1
        line["date6"] = date6
        line["date7"] = date7
        line["final_date"] = final_date
        line["htmldate"] = htmldate
        line["country"] = "Germany"
        line["level"] = state
        line["text_file_loc"] = "{}/{}.txt".format(text_dir, id)
        line['doctext'] = doctext

        
        # Printing processing status
        processed += 1
        print("processed: {}%".format((processed/total_count)*100))
        print("Current Time: ", datetime.now())

    # opening the output csv
    outputfile = open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/{}.csv".format(state), "w")
    
    # writing all the array lines into the csv file
    writer = csv.DictWriter(outputfile, fieldnames=["id", "country", "level", 'url', 'domain', 'date1', 'date2', 'date3', 'date4', 'date5', 'date5_1', 'date6', 'date7', 'htmldate', 'final_date', 'heading', 'agile_term', 'agile_context_pre', 'agile_context_post', 'digital_term', 'text_file_loc', 'doctext',])
    writer.writeheader()
    for id, line in array.items():
        writer.writerow(line)
    outputfile.close()  

In [4]:
#######Processing Statewise Germany

state_list = ["Federal"]
#, "Schleswig-Holstein","Bremen","Hamburg","Lower Saxony","Thuringia","Saxony-Anhalt","Saxony","Mecklenburg-Vorpommern","Berlin","Brandenburg","North Rhine-Westphalia","Rhineland-Palatinate","Saarland", "Hesse","Bavaria","Baden-Wuerttemberg"]

for state in state_list:
    print('processing ', state)
    process_state_DE(state)


ocessed: 8.287292817679557%
Current Time:  2020-04-19 12:36:13.877050
processing 9e925771-d3a3-3b40-add7-fde6e74d97f2
processed: 8.83977900552486%
Current Time:  2020-04-19 12:36:14.037286
processing f470ba9c-6aa9-39a0-86e5-d560d928cd16
processed: 9.392265193370166%
Current Time:  2020-04-19 12:36:14.195216
processing 9b2b3bc8-d57f-345b-a4cc-66acb4144e91
processed: 9.94475138121547%
Current Time:  2020-04-19 12:36:14.278774
processing 2233b421-720a-391b-b517-ae3a5233afe4
processed: 10.497237569060774%
Current Time:  2020-04-19 12:36:14.369322
processing 242e8fc5-7383-3e83-bd2c-a00b7ee9b877
processed: 11.049723756906078%
Current Time:  2020-04-19 12:36:14.464343
processing 635cbcf2-0fe2-39cd-91b5-d1ad79a8440d
processed: 11.602209944751381%
Current Time:  2020-04-19 12:36:14.587214
processing 014e2d2f-779e-3685-aac6-47230e4a7942
processed: 12.154696132596685%
Current Time:  2020-04-19 12:36:14.686421
processing 57879f15-b1f3-3b02-b989-3a40fce1a6cd
processed: 12.70718232044199%
Current Ti

In [5]:
# Adding Organisation Names based on domain FEDERAL

# specifying input and output file and paths
inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/Federal.csv"
outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/Federal_org.csv"

# file with all the uk subdomains (manual eddited to include subsubdomains)
domains_dict = pd.read_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/List_of_German_Domain_Names.csv")

# compiling regex to extract domains from url
domain_regex = re.compile(r'http(s?)://(?!.*http)(www\.)?([a-zA-Z0-9\-.]+)', re.IGNORECASE | re.UNICODE)

# defining the url extraction function
def extract(url):
    match = domain_regex.search(url)
    if match:
        match_domain = match.group(3)
        return '.'.join(match_domain.split('.')[-4:])
    else:
        print(url)
        return ''

# adding the domain / organisation names to the file
with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
    df = pd.read_csv(inFile)
    # getting the subdomain from 
    df['sub_domain'] = df.apply(lambda row: extract(row['url']), axis=1)
    # merging the file with the dataframe that has all the subdomains and organisation names
    df2 = pd.merge(df, domains_dict, left_on = 'sub_domain', right_on='sub_domain', how='left')
    # saving the file
    df2.to_csv(outFile)

In [6]:
#####Merging all German States
state_list = ["Federal_org", "Schleswig-Holstein","Bremen","Hamburg","Lower Saxony","Thuringia","Saxony-Anhalt","Saxony","Mecklenburg-Vorpommern","Berlin","Brandenburg","North Rhine-Westphalia","Rhineland-Palatinate","Saarland" ,"Hesse" ,"Bavaria","Baden-Wuerttemberg"]

state_df = {}

def reading_csv(state):
    inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/{}.csv".format(state)
    outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/Combined.csv"
    with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
        df = pd.read_csv(inFile)
    return df

combined_dataframe_Germany = None 

for state in state_list:
    temp_df = reading_csv(state)
    if combined_dataframe_Germany is None:
       combined_dataframe_Germany = temp_df
    else: 
        combined_dataframe_Germany = pd.concat([combined_dataframe_Germany,temp_df])

    combined_dataframe_Germany.to_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/Combined.csv")

In [7]:
# Cleaning the data and selecting only relevant pages based on the co-searchterms dictionary

inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/Combined.csv"
outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/Combined_cleaned.csv"

# loading the spacy model
nlp = spacy.load("de_core_news_sm")
# switching off irrelevant spacy functions
nlp.disable_pipes('tagger', 'ner')

# defining the keywords search function
def searchfunction(row):
    matchstring = ""
    count = 0
    for term in searchterms_de_eng_lemma:
        if term.lower() in row['doctext_lemma']:
            count += 1
            matchstring = matchstring + term.lower() + ", "
    row['co_searchterms_nr'] = count
    row['co_searchterms_matches'] = matchstring
    return row


# opening and cleaning / preprocessing dataframe
with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
    # reading the csv
    df = pd.read_csv(inFile)
    # sorting the values by final date
    df.sort_values(by=['final_date'])
    # dropping all sites which didn't yield any match for the agile term
    df.dropna(axis=0, how='any', thresh=None, subset=(['agile_term']), inplace=True)
    # dropping potential duplicates based on the heading and the context of the first agile term mention
    df.drop_duplicates(subset=(['heading', 'agile_context_pre', 'agile_context_post']), inplace=True)
    # changing htmldate to string for later processing
    df['htmldate'] = df['htmldate'].astype(str)
    # deleting sites with csv previews
    df = df[~df.url.str.endswith("csv/preview")]
    # deleting job search site entries
    df = df[~df.domain.str.contains("www.yojo.de")]
    # deleting search sites
    df = df[~df.url.str.contains('/search/')] 
    # deleting search results for sachsen
    df = df[~df.domain.str.contains('search.sachsen.de')] 
    # deleting terms related to agilis which is a lizard
    df = df[~df.agile_term.str.contains('agilis')]
    # deleting ticket counts
    df = df[~df.url.str.contains('/tickets/')] 
    # deleting hits related to agile horses which sprint
    df = df[~df.url.str.contains('animalfunfacts.net')] 
    # deleting wikipedia entries
    df = df[~df.url.str.contains('wikipedia.org')] 
    # deleting content where date has been wrongly classified and true date is none-attainable
    df = df[~df.htmldate.str.contains('1999-01-01')] 
    # deleting non-german content
    df = df[~df.url.str.contains('/FR/')] # deleting non-german content
    df = df[~df.url.str.contains('/fr/')] # deleting non-german content
    df = df[~df.url.str.contains( '/EN/'  )] # deleting non-german content
    df = df[~df.url.str.contains('/en/' )] # deleting non-german content
    df = df[~df.url.str.contains('/breg-en/' )] # deleting non-german content
    df = df[~df.url.str.contains('/breg-fr/' )] # deleting non-german content
    # deleting agile metioned in a tweet that occurs on multiple sites 
    df['doctext'] = df['doctext'].map(lambda x: x.strip('#Agile Development lessons learned from Gov.uk'))
    #deleting cases where no date has been found (htmldate attributes 2020-01-01 for no dates)
    df = df[df['final_date'] != '2020-01-01 00:00:00']
    #deleting all cases from 2020 because the analysis focuses on everything up until 2019
    df = df[df['final_date'] < '2020-01-01 00:00:00']
    # changing the final_date column to a pandas datetime object
    df['final_date'] = pd.to_datetime(df['final_date'])

    # lemmatizing the text
    lemma = []
    for doc in nlp.pipe(df['doctext'].astype('unicode').values, batch_size=50, n_threads=3):
        if doc.is_parsed:
            lemma.append([n.lemma_ for n in doc])
        else:
            lemma.append(None)

    # storing the lemmas in a new column 
    df['doctext_lemma'] = lemma
    
    #making lemmas lowercase
    df['doctext_lemma'] = df['doctext_lemma'].map(lambda lemmas: [x.lower() for x in lemmas])
    
    #joining list of lemmas into one string agein, so it becomes matchable by the searchfunction
    df['doctext_lemma'] = df['doctext_lemma'].map(lambda lemmas: " ".join(lemmas))
 
    #applying the keyword search function 
    df = df.apply(searchfunction, axis = 1)

    # delete everything that didn't yield a co-searchterm match
    df = df[df['co_searchterms_nr'] != 0 ]
    df.to_csv(outFile)

In [6]:
'''
# Adding subdomain Names based on domain

# specifying input and output file and paths
inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/Combined_cleaned.csv"
outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/Combined_cleaned_subdomains.csv"


# compiling regex to extract domains from url (also in the case of a site on national archives)
domain_regex = re.compile(r'http(s?)://(?!.*http)(www\.)?([a-zA-Z0-9\-.]+)', re.IGNORECASE | re.UNICODE)
alternate_url_regex = re.compile(r'\.de/([\w%-]+)(/)?([\w%-]+)?(/)?', re.IGNORECASE | re.UNICODE)
#alternate_sub_regex = re.compile()

# defining the url extraction function
def extract1(url):
    match = domain_regex.search(url)
    if match:
        match_domain = match.group(3)
        return '.'.join(match_domain.split('.')[-4:])
    else:
        print(url)
        return ''
#defining second extraction function
def extract2(url):
    match = alternate_url_regex.search(url)
    if match:
        match= match.group(1)
        return match
    else:
        print(url)
        return ""
#defining third extraction function
def extract3(url):
    match = alternate_url_regex.search(url)
    if match:
        match = match.group(3)
        return match
    else:
        print(url)
        return ""
# defining fourth url extraction function
def extract4(url):
    match = domain_regex.search(url)
    if match:
        match_domain = match.group(3)
        return match_domain.split('.')[0]
    else:
        print(url)
        return ''

# adding the domain / organisation names to the file
with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
    df = pd.read_csv(inFile)
    # getting the subdomain from 
    df['sub_domain'] = df.apply(lambda row: extract1(row['url']), axis=1)
    df['sub_domain_name'] = df.apply(lambda row: extract4(row['url']), axis=1)
    df['url_match_1'] = df.apply(lambda row: extract2(row['url']), axis=1)
    df['url_match_2'] = df.apply(lambda row: extract3(row['url']), axis=1)
    # merging the file with the dataframe that has all the subdomains and organisation names
    # saving the file
    df.to_csv(outFile)

https://de.m.wikipedia.org/w/index.php?title=PRINCE2
https://de.m.wikipedia.org/w/index.php?title=PRINCE2


In [77]:
###### COMBINING ALL DATAFRAMES ######

In [8]:
df_germany =  pd.read_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Germany/Combined_cleaned.csv")
df_uk = pd.read_csv('/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/UK/General_cleaned_org.csv')
df_master = pd.concat([df_germany,df_uk])

df_master.to_csv('/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Analysis/2_Data_Preprocessing/DATA/CSVs/Master_File.csv')